In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy.stats import norm
import random
import networkx as nx
from itertools import product

In [10]:
class vehicle():
    def __init__(self, tipp, plate, spot, base):
        self.stat = "wh"
        self.returned = True
        self.speed = 60
        self.spot_factor = 1
        self.plate = plate
        self.spot = spot
        self.base = base
        self.base_index = self.base.index
        self.tip = tipp
        if tipp == "T":   self.cap = 33;        self.cost = 2.81; self.rem_cap = 33
        else:             self.cap = 18;        self.cost = 2.48; self.rem_cap = 18
        
    def set_trip_cost_spot(self, destination, time, q, data_trip):  #CLEAR
        distance = data_trip["distance_matrix"][self.base_index, destination.index]
        duration = (distance / self.speed) / 24
        self.return_time = time + duration*self.spot_factor
        trip_cost = distance*self.cost
        show_ship(self, time, destination, duration, distance, trip_cost, q)
        return trip_cost
    def set_trip_cost_ded(self, distance, time, destination, route_s, customers, fleet ):  #CLEAR
        self.returned = False
        duration = (distance / self.speed) / 24
        self.return_time = time + duration*self.spot_factor
        trip_cost = distance*self.cost
        show_ship_ded(self,route_s, customers)
        if self.tipp == "T":
            fleet.v_left(self); fleet.avail_check()
        return [fleet, trip_cost]
    
class make_fleet():
    def __init__(self, vehicles):
        self.v = vehicles;        self.available_truck=0;        self.available_lorry=0
        for vehicle in self.v:
            if vehicle.tip == "T":          self.available_truck+=1
            if vehicle.tip == "K":          self.available_lorry+=1
        self.havetruck = (self.available_truck > 0 )
        self.havelorry = (self.available_lorry > 0 )
    def v_left(self, v):
        if v.tip == "T": self.available_truck-=1;        
        else:  self.available_lorry-=1    
    def v_return(self, v):
        if v.tip == "T":            self.available_truck+=1
        else:                       self.available_lorry+=1
    def avail_check(self):
        self.havetruck = (self.available_truck > 0 )
        self.havelorry = (self.available_lorry > 0 )
        
class location():
    def __init__(self,name, tip, ind):
        self.name=name;        self.tip=tip;     self.index = ind; self.serviceLevel = []
        self.acc_orders = [0,0,0,0,0,0,0];  self.days_since_ship = 0; self.passive = True
        self.tr_o=0;
    def shipment_received(self):
        self.days_since_ship = 0
    def dayPassed(self):
        self.days_since_ship += 1
    def w_o_truck(self,tr_dat):
        for i in range(len(tr_dat['Location'])):
            if tr_dat['Location'][i]==self.name:
                self.tr_o=tr_dat['Without truck'][i]

class order_sum():
    def __init__(self, init, place, amnt):
        self.dest = place;     self.q = amnt;       self.init = init

class order():
    status = "new"
    def __init__(self, place, prod, amnt, init):
        self.dest = place;        self.x = place.x;     self.y = place.y
        self.product = prod;      self.q = amnt;        self.init = init
class output_statistic():
    def __init__(self, quantities, veh, isinbound, distancesArr, totalemptydist, totDist):
        fillOfTravel = fill_rate(quantities, veh.cap, isinbound)
        fillbin = fill_rate_binary(quantities, isinbound)
        self.mean_fill = np.inner(fillOfTravel, distancesArr) / (sum(distancesArr)+.1)
        self.mean_fill_bin = np.inner(fillbin, distancesArr) / (sum(distancesArr)+.1)
        self.emptyDistRate = totalemptydist / (totDist+.1)


def show_ship(v, time, dest, dur, dist, cost,q):
    print(str(v.base.name)+ " --> "+str(dest.name) + 
          " Distance: " + str("%0.2f" % dist) + " km" + " Cost: " + "₺"+str("%0.2f" % (cost)) 
          + " Time: " +str("%0.2f" % time))
    print("Vehicle: ", str(v.plate), " Spot?: ", v.spot ," Quantity: ", "%0.2f" % q)
    #print("\n####### NEXT SHIPMENT #######")

def show_ship_ded(v, route, locations):
    str1 = v.plate + " " + v.base.name
    for key in route:
        if key != v.base_index and key != "Distances" and key!="Total Load":
            str1 += "-> " + locations[key].name + "("+ route[key] +"p) "
        str1 += route[Distances] +" km"
    print(str1)

def assign_vehicle(wh, order, fleet, time):
    q = order.q #take order's quantity
    assigned = False
    no_vehicle=True
    order.dest.order_not_sat(order.dest, order)
    for vehicle in fleet.v:
        if vehicle.stat == "wh" and not assigned:
            novehicle = False
            if  q <= vehicle.cap:
                assigned = True
                vehicle.stat = "ship"
                new_shipment = shipment(wh, vehicle, order, time)
                vehicle.set_trip(new_shipment)
                vehicle.returned = False
                fleet.v_left(vehicle)
                order.dest.order_sat(q)
            else:
                if vehicle.tip == "T":
                    assigned = True
                    vehicle.stat = "ship"
                    new_shipment = shipment(wh, vehicle, order, time)
                    vehicle.set_trip(new_shipment)
                    vehicle.returned = False
                    fleet.v_left(vehicle)
                    order.dest.order_sat(vehicle.cap)
    if not assigned:
        for vehicle in fleet.v:
            if vehicle.stat == "wh":
                assigned = True
                vehicle.stat = "ship"
                new_shipment = shipment(wh, vehicle, order, time)
                vehicle.set_trip(new_shipment)
                vehicle.returned = False
                fleet.v_left(vehicle)
                order.dest.order_sat(vehicle.cap)

    if assigned:
        show_ship(new_shipment, time)
        return new_shipment 
    else:
        print("###### no vehicle found for delivery :( #######")

        
        
#Reads the distance matrix        
def load_distances (data_load, dist_df):
    total_locs = dist_df.shape[0]  #number of rows
    dis = np.zeros((total_locs,total_locs)) #empty square matrix with  total locations 
    for i in range (total_locs):
        dis[i] = dist_df.iloc[i][1:total_locs+1] #i th row will include all the distances in the matrix row
    data_load['distance_matrix'] = dis
    return data_load


def load_distances_dum (data_load, dist_df):
    total_locs = dist_df.shape[0]
    dis = np.zeros((total_locs,total_locs))
    for i in range (total_locs):
        dis[i] = dist_df.iloc[i][1:total_locs+2]
    data_load['distance_matrix'] = dis
    return data_load
def load_demands (data_dem, customers, i):
    dem = []
    for loc in customers:
        dem.append(loc.acc_orders[i])
    data_dem['demands'] = dem
    return data_dem
def load_caps (data_cap, fleet):
    caps = []
    for vehicle in fleet.v:
        if vehicle.returned:
            caps.append(vehicle.cap)
        else:
            caps.append(0)
    data_cap['vehicle_capacities'] = caps
    return data_cap



#Defining the benefits for inbound hybrid
def load_benefits_start_inbnd(depots,hyb):
    r=[7 if hyb else 3][0]
    benefits=[[] for i in range(r)]
    cur_max = [0 for i in range(r)]
    cur_max_ind = [-1 for i in range(r)]
    for i in depots:
        for k in range(len(customers_ic)): benefits[i].append(0)
        #inbound
        for j in customers_ic:
            k=j.index
            if 0 and (k in [3,4,5]):    #while there are vehicles at plants no need for these conditions therefore there are 0s in ifs
                benefits[i][k]+= 0.7*1000*int(j.acc_orders[i]) / (data_geb_ic["distance_matrix"][i][k] + .1 ) +5;  ##BENEFIT SHOULD BE RECONSIDERED
            elif 0 and (k==6):
                benefits[i][k]+= 0.7*1000*int(j.acc_orders[i]) / (data_geb_ic["distance_matrix"][i][k] + .1 ) +10;  ##BENEFIT SHOULD BE RECONSIDERED
            else:
                k=j.index
                benefits[i][k]+= 0.7*1000*int(j.acc_orders[i]) / (data_geb_ic["distance_matrix"][i][k] + .1 ) 
            
            if dist_df_ic.iloc[k]["belongs"] == i:
                benefits[i][k] *= 2
            else:
                benefits[i][k] *=0.5
        cur_max[i] = np.max(benefits[i])
        cur_max_ind[i] = int(np.argmax(benefits[i]))
    chosen_wh_ind = int(np.argmax(cur_max)) #gives the selected warehouse
    chosen_dest_ind = cur_max_ind[chosen_wh_ind] #gives the selected customer location 
    return [cur_max[chosen_wh_ind], chosen_wh_ind, chosen_dest_ind]



def load_benefits_start_spot_hyb(depots,hyb):
    r=[7 if hyb else 3][0]
    benefits=[[] for i in range(r)]
    cur_max = [0 for i in range(r)]
    cur_max_ind = [-1 for i in range(r)]
    for i in depots:
        for k in range(len(customers_dis)): benefits[i].append(0)
        #outbound
        for locat in customers_dis:
            k = locat.index
            if k not in [3,4,5,6]:
                benefits[i][k] +=  1000 * int(locat.acc_orders[i]) / (data_geb_dis["distance_matrix"][wh_ind_outbnd[i]][k] + 0.1)
                if dist_df.iloc[k]["belongs"] == i:
                    benefits[i][k] *= 2 #####################
                else:
                    benefits[i][k] *= .5 ####################
                
                if locat.tip=='zm': benefits[i][k] *= 1.2 + .5 *locat.days_since_ship
                else: benefits[i][k] *= 1   + .1 *locat.days_since_ship
            else:
                benefits[i][k]+=0
        if i > 2 : benefits[i] = [2*q for q in benefits[i]]
        cur_max[i] = np.max(benefits[i])
        cur_max_ind[i] = int(np.argmax(benefits[i]))
    #print(cur_max)
    chosen_wh_ind = int(np.argmax(cur_max))
    chosen_dest_ind = cur_max_ind[chosen_wh_ind]
    return [cur_max[chosen_wh_ind], chosen_wh_ind, chosen_dest_ind]



def find_vehicle_inbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot):
    i = chosen_wh_ind
    q = customers_ic[chosen_dest_ind].acc_orders[i]
    vehiclefound=False
    limitForTruck = [15 if lookForSpot else 8][0]#################
    if q > limitForTruck or (chosen_dest_ind in [0,1,2,3,4,5,6,7,8,9]): ########################
        if fleets[i].havetruck:  desired = "T"
        else: desired = "K"
    else: desired = "K"

    if lookForSpot:
        for v in fleets[i].v:
            if v.returned and v.tip == desired and v.spot==True and not vehiclefound:
                v.returned = False;
                fleets[i].v_left(v); fleets[i].avail_check()
                vehiclefound=True
                return v
        if not vehiclefound: return 0
    else:
        for v in fleets[i].v:
            if v.returned and v.tip == desired and v.spot==False and not vehiclefound:
                v.returned = False;
                fleets[i].v_left(v); fleets[i].avail_check()
                vehiclefound=True
                return v
        if not vehiclefound: return 0

def find_vehicle_outbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot):
    i = chosen_wh_ind
    j = chosen_dest_ind
    q = customers_dis[chosen_dest_ind].acc_orders[i]
    vehiclefound=False
    limitForTruck = [0 if lookForSpot else 8][0]
    for locat in customers_dis:
      if locat.index==j:
        tr_avail=locat.tr_o
    if q > limitForTruck: 
        if fleets[i].havetruck and (not tr_avail):  desired = "T"
        else: desired = "K"
    else: desired = "K"

    for v in fleets[i].v:
        if v.returned and v.tip == desired and not vehiclefound:
            v.returned = False;
            fleets[i].v_left(v); fleets[i].avail_check()
            vehiclefound=True
            return v
    if not vehiclefound: return 0


#Arragements in benefits and returns the lindex of max benefit
def load_benefits_route_inbnd(cur_loc, vhc, routeSoFar, is_spot):
    a=1;
    wh_i = vhc.base_index
    benefits=[]    
    cur_max = []
    cur_max_ind = []
    i = vhc.base_index
    if (cur_loc in [3,4,5,6,7,8,9]) or (is_spot):
        return "fail"
    else:
        for locat in customers_ic:
            if locat.index not in routeSoFar:
                benefits.append( 1000 * np.min([int(locat.acc_orders[wh_i]), vhc.rem_cap]) / ((data_geb_ic["distance_matrix"][cur_loc][locat.index]) + 0.01) )
                if locat.index  in [0,1,2,3,4,5,6] and not is_spot:
                    if locat.index in [3,4,5,6]:
                      benefits[locat.index]+=.7*20
                    else:
                      j_cus = locat.index
                      if i in [0,1,2]:
                          benefits[locat.index] =  1000 * customers_ic[i+7].acc_orders[j_cus] / ((data_geb_ic["distance_matrix"][cur_loc][locat.index])**1.2 + 0.01)
                          benefits[locat.index] *= (2*vhc.cap - vhc.rem_cap)/vhc.cap
                      else:
                          benefits[locat.index] =  1000 * customers_ic[i].acc_orders[j_cus] / ((data_geb_ic["distance_matrix"][cur_loc][locat.index])**1.2 + 0.01)
                          benefits[locat.index] *= (2*vhc.cap - vhc.rem_cap)/vhc.cap

            else:
                benefits.append(0)
        cur_max = (np.max(benefits))
        cur_max_ind = (int(np.argmax(benefits)))
        if cur_max > thresholds[wh_i]: ##################
            return cur_max_ind
        else: return "fail"

def load_benefits_route_outbnd(cur_loc, vhc, routeSoFar, is_spot):
    wh_i = vhc.base_index
    benefits=[]    
    cur_max = []
    cur_max_ind = []
    i = vhc.base_index
    if cur_loc in [0,1,2]:
        return "fail"
    else:
        for locat in customers_dis:
            if (vhc.tip == 'T'): 
              if locat.index not in routeSoFar:
                  if locat.tr_o:
                        benefits.append(0);
                  else:
                      benefits.append( 1000 * np.min([int(locat.acc_orders[wh_i]), vhc.rem_cap]) / ((data_geb_dis["distance_matrix"][cur_loc][locat.index]) + 0.01) )
                      if dist_df.iloc[locat.index]["belongs"] == i:
                          benefits[locat.index] *= 2  ####################
                      else:
                          benefits[locat.index] *= .5 ####################
              else:
                  benefits.append(0)
            else:
              if locat.index not in routeSoFar:
                  benefits.append( 1000 * np.min([int(locat.acc_orders[wh_i]), vhc.rem_cap]) / ((data_geb_dis["distance_matrix"][cur_loc][locat.index]) + 0.01) )
                  if dist_df.iloc[locat.index]["belongs"] == i:
                      benefits[locat.index] *= 2  ####################
                  else:
                      benefits[locat.index] *= .5 ####################
              else:
                  benefits.append(0)

        cur_max = (np.max(benefits))
        #print(cur_max)
        cur_max_ind = (int(np.argmax(benefits)))
        if cur_max > thresholds[wh_i]: ##################
            return cur_max_ind
        else: return "fail"
      

#Şaheser part for hybrid ic       
def find_route_inbnd_ded(strprint, v, current_loc_ind, spot):
    wh_index = v.base_index
    routeLimit =[3 if spot else 3][0]
    routes = []
    quants = []
    distances= []
    totalEmptyDistance = 0
    routes.append(v.base_index)
    customers_ic[v.base_index].shipment_received()
    distance = data_geb_ic["distance_matrix"][v.base_index][current_loc_ind]#going
    total_distance =  distance
    distances.append(distance)
    if current_loc_ind in [0,1,2,3,4,5,6]: totalEmptyDistance += distance
    routes.append(current_loc_ind)
    customers_ic[current_loc_ind].shipment_received()
    next_target = current_loc_ind
    inbound = False
    while True: 
        #inbound
        if next_target in [0,1,2,3,4,5,6]:
            j_cus = next_target
            quant = v.cap
            quants.append(0)
            if next_target in [0,1,2]:
                customers_ic[v.base_index+7].acc_orders[j_cus] -= quant
            else:
                customers_ic[v.base_index].acc_orders[j_cus] -= quant
            strprint += " -> " + customers_ic[next_target].name + "(given:" + "0" + ")"

        elif next_target != "fail": 
            quant = int(np.min([v.rem_cap,   customers_ic[next_target].acc_orders[wh_index]]))
            quants.append(quant)
            v.rem_cap -= quant
            customers_ic[next_target].acc_orders[wh_index] -= quant 
            strprint += " -> " + customers_ic[next_target].name + "(given:" + str(quant) +")"

        current_loc_ind = next_target
        if len(routes) >= routeLimit:  ##################### route destination number
            next_target= "fail"
        else:
            next_target = load_benefits_route_inbnd(next_target, v, routes, spot)


        if next_target != "fail":
            distance = data_geb_ic["distance_matrix"][current_loc_ind][next_target]#return
            total_distance +=  distance
            distances.append(distance)
            routes.append(next_target)
            customers_ic[next_target].shipment_received()
            if next_target in [0,1,2, 3, 4, 5, 6]: totalEmptyDistance += distance
        if (next_target == "fail"):
            if not (spot): 
                quants.append(0)
                strprint += " -> " + customers_ic[v.base_index].name
                distance = data_geb_ic["distance_matrix"][current_loc_ind][v.base_index]#going
                total_distance +=  distance
                distances.append(distance)
                routes.append(v.base_index)
                if current_loc_ind not in [0,1,2,3,4,5,6]: totalEmptyDistance += distance
            break
    st = output_statistic(quants, v, inbound, distances, totalEmptyDistance, total_distance)
    return [routes, total_distance, strprint, st]

#Şaheser part        
def find_route_outbnd_spt(strprint, v, current_loc_ind, spot):
    wh_index = v.base_index
    routeLimit =3
    routes = []
    quants = []
    distances= []
    totalEmptyDistance = 0
    routes.append(v.base_index)
    customers_dis[v.base_index].shipment_received()
    distance = data_geb_dis["distance_matrix"][v.base_index][current_loc_ind]#going
    total_distance =  distance
    distances.append(distance)
    if current_loc_ind in [0,1,2]: totalEmptyDistance += distance
    routes.append(current_loc_ind)
    customers_dis[current_loc_ind].shipment_received()
    next_target = current_loc_ind
    inbound = False
    while True:
        if next_target != "fail": 
            quant = int(np.min([v.rem_cap,   customers_dis[next_target].acc_orders[wh_index]]))
            quants.append(quant)
            v.rem_cap -= quant
            customers_dis[next_target].acc_orders[wh_index] -= quant 
            strprint += " -> " + customers_dis[next_target].name + "(given:" + str(quant) +")"

        current_loc_ind = next_target
        if len(routes) >= routeLimit:  ##################### route destination number
            next_target= "fail"
        else:
            next_target = load_benefits_route_outbnd(next_target, v, routes, spot)

        if next_target != "fail":
            distance = data_geb_dis["distance_matrix"][current_loc_ind][next_target]#return
            total_distance +=  distance
            distances.append(distance)
            routes.append(next_target)
            customers_dis[next_target].shipment_received()
            if next_target in [0,1,2]: totalEmptyDistance += distance
        if next_target == "fail":
            break
    st = output_statistic(quants, v, inbound, distances, totalEmptyDistance, total_distance)
    return [routes, total_distance, strprint, st]


def fill_rate(quant_arr, capacity, inb):
    fillOnRoad =[]
    start_load = sum(quant_arr)
    curLoad=start_load
    for r in range(len(quant_arr)):
        fillOnRoad.append(curLoad/capacity)
        curLoad -= quant_arr[r]
    if inb:
        fillOnRoad.append(1)
    return fillOnRoad

def fill_rate_binary(quant_arr,inb):
    fillOnRoad =[]
    start_load = sum(quant_arr)
    curLoad=start_load
    for r in range(len(quant_arr)):
        if curLoad > 0:
            fillOnRoad.append(1)
        else:
            fillOnRoad.append(0)
        curLoad -= quant_arr[r]
    if inb:
        fillOnRoad.append(1)
    return fillOnRoad


def prepare_fleet_inbnd(trucks, lorries,trcks_spt,lrs_spt):
    for i in range(7):
        trucknumber=1;   lorrynumber=1;   fleetAtHand=[];   baseind = i
        Strucknumber=1;  Slorrynumber=1;
        while trucks[i] + lorries[i] > 0:
            if trucks[i] >0:
                name = "34 T 0" + str(trucknumber)
                fleetAtHand.append(vehicle("T", name, spot=False, base=customers_ic[i]))
                trucknumber +=1
                trucks[i] -=1
            if lorries[i] >0:
                name = "34 K 0" + str(lorrynumber)
                fleetAtHand.append(vehicle("K", name, spot=False, base=customers_ic[i]))
                lorrynumber+=1
                lorries[i] -=1
        while trcks_spt[i] + lrs_spt[i] > 0:
            if trcks_spt[i] >0:
                name = "34S T 0" + str(Strucknumber)
                fleetAtHand.append(vehicle("T", name, spot=True, base=customers_ic[i]))
                Strucknumber +=1
                trcks_spt[i] -=1
            if lrs_spt[i] >0:
                name = "34S K 0" + str(Slorrynumber)
                fleetAtHand.append(vehicle("K", name, spot=True, base=customers_ic[i]))
                Slorrynumber+=1
                lrs_spt[i] -=1 
        if i ==0: fleet_geb = make_fleet(fleetAtHand)
        if i ==1: fleet_nev = make_fleet(fleetAtHand)
        if i ==2: fleet_ala = make_fleet(fleetAtHand)
        if i ==3: fleet_bil = make_fleet(fleetAtHand)
        if i ==4: fleet_sar = make_fleet(fleetAtHand)
        if i ==5: fleet_ela = make_fleet(fleetAtHand)
        if i ==6: fleet_mer = make_fleet(fleetAtHand)
    fleets = [fleet_geb, fleet_nev, fleet_ala,fleet_bil,fleet_sar,fleet_ela,fleet_mer]
    return fleets

def prepare_fleet_otbnd(trucks, lorries):
    for i in range(7):
        trucknumber=1;   lorrynumber=1;   fleetAtHand=[];   baseind = i
        while trucks[i] + lorries[i] > 0:
            if trucks[i] >0:
                name = "34 T 0" + str(trucknumber)
                fleetAtHand.append(vehicle("T", name, spot=True, base=customers_dis[i]))
                trucknumber +=1
                trucks[i] -=1
            if lorries[i] >0:
                name = "34 K 0" + str(lorrynumber)
                fleetAtHand.append(vehicle("K", name, spot=True, base=customers_dis[i]))
                lorrynumber+=1
                lorries[i] -=1
        if i ==0: fleet_geb = make_fleet(fleetAtHand)
        if i ==1: fleet_nev = make_fleet(fleetAtHand)
        if i ==2: fleet_ala = make_fleet(fleetAtHand)
        if i ==3: fleet_bil = make_fleet(fleetAtHand)
        if i ==4: fleet_sar = make_fleet(fleetAtHand)
        if i ==5: fleet_ela = make_fleet(fleetAtHand)
        if i ==6: fleet_mer = make_fleet(fleetAtHand)
    fleets = [fleet_geb, fleet_nev, fleet_ala,fleet_bil,fleet_sar,fleet_ela,fleet_mer]
    return fleets


def spot_cost_calculator_hyb_inb(route, dist, vv, c_routes, c_costs):
    for j in range(len(c_routes)):
        r = c_routes[j]
        if route[0:2] in c_routes and len(route) < 3:
            return [c_costs[j],0]
            break
    theCost = (dist + data_geb_ic["distance_matrix"][route[-1]][vv.base_index])* vv.cost
    return [theCost, data_geb_ic["distance_matrix"][route[-1]][vv.base_index]*vv.cost ]


def spot_cost_calculator_hyb(route, dist, vv, c_routes, c_costs):
    for j in range(len(c_routes)):
        r = c_routes[j]
        if route[0:2] in c_routes and len(route) < 3:
            return [c_costs[j],0]
            break
    theCost = (dist + data_geb_dis["distance_matrix"][route[-1]][vv.base_index])* vv.cost
    return [theCost, data_geb_dis["distance_matrix"][route[-1]][vv.base_index]*vv.cost ]

def route_appender(lst,route):
    lst.append(route)
    return lst

def set_route_for_spot(r , vtype):
    return 0

In [11]:
#READING EXCEL FILES


woTr=pd.read_excel(r"/content/drive/MyDrive/MEYYO/without_truck.xlsx")
dist_df = pd.read_excel(r"/content/drive/MyDrive/MEYYO/distances (2).xlsx")
dist_df_ic=pd.read_excel(r"/content/drive/MyDrive/MEYYO/distances_ic.xlsx")
loc_type_df = pd.read_excel(r"/content/drive/MyDrive/MEYYO/musteri_tip.xlsx")

loc_type_df_dis=pd.read_excel(r"/content/drive/MyDrive/MEYYO/musteri_tip_dis.xlsx")
dist_df_dis=pd.read_excel(r"/content/drive/MyDrive/MEYYO/distances_dis.xlsx")

#data_geb becomes full distance matrix 
data_geb_ic={}; data_geb_dis={}
data_geb_ic=load_distances(data_geb_ic,dist_df_ic)
data_geb_dis=load_distances(data_geb_dis,dist_df_dis)

customers = []
for i in range(dist_df.shape[0]):
    name = str(dist_df.iloc[i]["FROM/TO"]) #names are taken from distances excel
    if loc_type_df.loc[loc_type_df.place == name].empty: 
        t = "na"
        if i in [168,169,170]:
            t= "wh"
    else:
        t=loc_type_df.loc[loc_type_df.place == name].iloc[0].tip
    customers.append(location(str(dist_df.iloc[i]["FROM/TO"]), t, i)) #assigns name,type,index to each point


customers_ic=[]
for i in range(dist_df_ic.shape[0]):
  name = str(dist_df_ic.iloc[i]["FROM/TO"]) #inbound names are taken from distances excel
  if i in [7,8,9]:
    t="wh"
  else:
    t="factory" 
  customers_ic.append(location(str(dist_df_ic.iloc[i]["FROM/TO"]), t, i)) #assigns name,type,index to each point
  

customers_dis=[]
for i in range(dist_df_dis.shape[0]):
  name=str(dist_df_dis.iloc[i]["FROM/TO"])
  if loc_type_df_dis.loc[loc_type_df.place == name].empty:
    t="na"
  else:
    t=loc_type_df_dis.loc[loc_type_df_dis.place == name].iloc[0].tip
  customers_dis.append(location(str(dist_df_dis.iloc[i]["FROM/TO"]), t, i))
  

  

#BİLECİK ARAÇLARI
vehicles_bil = [   #SET UP VEHICLES (tip, plaka, spot mu?, merkez depo)
            vehicle("T", "34 STB 1", spot=True, base = customers[164]),           vehicle("T", "34 STB 2", spot=True, base = customers[164]),
            vehicle("T", "34 STB 3", spot=True, base = customers[164]),           vehicle("T", "34 STB 4", spot=True, base = customers[164]),
            vehicle("K", "34 SKB 5", spot=True, base = customers[164]),           vehicle("K", "34 SKB 6", spot=True, base = customers[164]),
]
fleet_bil = make_fleet(vehicles_bil) #make the fleet with vehicle array

#MERSİN ARAÇLARI
vehicles_mer = [   #SET UP VEHICLES (tip, plaka, spot mu?, merkez depo)
            vehicle("T", "34 STM 1", spot=True, base = customers[165]),           vehicle("T", "34 STM 2", spot=True, base = customers[165]),
            vehicle("T", "34 STM 3", spot=True, base = customers[165]),           vehicle("K", "34 SKM 4", spot=True, base = customers[165]),
]
fleet_mer = make_fleet(vehicles_mer) 

#ŞARKÖY ARAÇLARI
vehicles_sar = [   #SET UP VEHICLES (tip, plaka, spot mu?, merkez depo)
            vehicle("T", "34 STS 1", spot=True, base = customers[163]),           vehicle("T", "34 STS 2", spot=True, base = customers[163]),
            vehicle("T", "34 STS 3", spot=True, base = customers[163]),           vehicle("K", "34 SKS 4", spot=True, base = customers[163]),
]
fleet_sar = make_fleet(vehicles_sar) #make the fleet with vehicle array

#ELAZIĞ ARAÇLARI
vehicles_ela = [   #SET UP VEHICLES (tip, plaka, spot mu?, merkez depo)
            vehicle("T", "34 STE 1", spot=True, base = customers[166]),           vehicle("T", "34 STE 2", spot=True, base = customers[166]),
            vehicle("T", "34 STE 3", spot=True, base = customers[166]),           vehicle("K", "34 SKE 4", spot=True, base = customers[166]),
]
fleet_ela = make_fleet(vehicles_ela) #make the fleet with vehicle array

#GEBZE ARAÇLARI
vehicles_geb = [   #SET UP VEHICLES (tip, plaka, spot mu?, merkez depo)
            vehicle("T", "34 DTG 1", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 2", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 3", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 4", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 5", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 6", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 7", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 8", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 9", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 10", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 11", spot=False, base = customers[0]),
            vehicle("T", "34 DTG 12", spot=False, base = customers[0]),
            ###
            vehicle("K", "34 DKG 12", spot=False, base = customers[0]),
            vehicle("K", "34 DKG 12", spot=False, base = customers[0]),
            ]
fleet_geb = make_fleet(vehicles_geb) #make the fleet with vehicle array

#NEVŞEHİR ARAÇLARI
vehicles_nev = [   #SET UP VEHICLES (tip, plaka, spot mu?, merkez depo)
            vehicle("T", "34 DTN 1", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 2", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 3", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 4", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 5", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 6", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 7", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 8", spot=False, base = customers[1]),
            vehicle("T", "34 DTN 9", spot=False, base = customers[1]),
            ###
            vehicle("K", "34 DKN 13", spot=False, base = customers[1]),
            vehicle("K", "34 DKN 13", spot=False, base = customers[1]),
            vehicle("K", "34 DKN 13", spot=False, base = customers[1]),
            vehicle("K", "34 DKN 13", spot=False, base = customers[1]),
            vehicle("K", "34 DKN 13", spot=False, base = customers[1]),
            vehicle("K", "34 DKN 13", spot=False, base = customers[1])
            ]
fleet_nev = make_fleet(vehicles_nev)

#ALAŞEHİR ARAÇLARI
vehicles_ala = [   #SET UP VEHICLES (tip, plaka, spot mu?, merkez depo)
            vehicle("T", "34 DTA 1", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 2", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 3", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 4", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 5", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 6", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 7", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 8", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 9", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 10", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 11", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 12", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 13", spot=False, base = customers[2]),
            vehicle("T", "34 DTA 14", spot=False, base = customers[2]),
            #vehicle("T", "34 DTA 15", spot=False, base = customers[2]),
            ###
            vehicle("K", "34 DKA 13", spot=False, base = customers[2]),
            vehicle("K", "34 DKA 13", spot=False, base = customers[2]),
            vehicle("K", "34 DKA 13", spot=False, base = customers[2]),
            vehicle("K", "34 DKA 13", spot=False, base = customers[2]),
            vehicle("K", "34 DKA 13", spot=False, base = customers[2]),
            vehicle("K", "34 DKA 13", spot=False, base = customers[2]),
            vehicle("K", "34 DKA 13", spot=False, base = customers[2]),
            #following are extra on top of exising
            ]
fleet_ala = make_fleet(vehicles_ala)

#fleets = [fleet_geb, fleet_nev, fleet_ala, fleet_bil, fleet_sar, fleet_ela, fleet_mer]

#By giving the numbers explicitly, the vehicles are generated in the prepare_fleet function
trks=[30,30,30,8,11,11,6]; lrs=[12,16,12,5,4,4,4];
pln = "hybrd_spot_simple_heur"  #or hybrd_spot_simple_heur
if pln== "hybrd_ded_simple_heur" :
    fleets = prepare_fleet_inbnd(trucks=trks, lorries = lrs, 
                                 trcks_spt=[50,50,50,50,50,50,50],lrs_spt=[50,50,50,50,50,50,50])
else:
    fleets = prepare_fleet_otbnd(trucks=trks, lorries = lrs)

demand_period = 1
demands_oct = pd.read_excel(r"/content/drive/MyDrive/MEYYO/totaldemand_new.xlsm")

demands_oct["norm_periodic"] = demands_oct.norm_demand / demands_oct.norm_count



In [12]:
#MAIN FUNCTION

plan_for = pln #change this part to select spot and dedicated scenarios
if plan_for=="hybrd_ded_simple_heur" :
    demand_plan = "frequency_inbnd" #Currently we only use this frequency based demand. Previously there were other options.
else:
    demand_plan = "frequency_outbnd"
hyb_inb=True;
fab_spot=True;
maxdays = 30 #max length of the simulation in days
time_increment = .25  #how much time will pass each iteration in days
bound_with_truck = 15
bound_with_lorry = 10
exit_points = [7 if hyb_inb else 3][0] #if the selection is based on dedicated then it will asign costs to 3 points o/w 7 points
total_cost = [0 for q in range(exit_points)]
totalReturnCost = [0 for q in range(exit_points)]
all_distances = [[] for q in range(exit_points)]
all_empty_distances = [[] for q in range(exit_points)]
all_mean_fill = [[] for q in range(exit_points)]
all_mean_fill_bin = [[] for q in range(exit_points)]
vehicle_avail = [[] for q in range(exit_points)]          #vehicle availability array started as empty
unsat_dem    = [[] for q in range(exit_points)]
if plan_for=="hybrd_ded_simple_heur" :
    contracted_routes = [[2,7], [1,7],[6,8], [1,9]]
else:
    contracted_routes = [[0,0], [0,0],[0,0], [0,0]]
contracted_costs = [2734, 3092, 2925, 3786]
wh_ind = [0,1,2,3,4,5,6]
wh_ind_outbnd=[0,1,2,3,4,5,6]
o=0; tot_route=[];
for time in np.linspace(0,maxdays,int(1+ maxdays / time_increment)): #iterates through time points according to parameters above
    if time%1==0:
        print("\n### DAY ", int(time+1),"dedike", " MORNING ###")
    elif time%1==0.25:
        print("\n### DAY ", int(time+1),"dedike", " NOON ###")
    
    if time%1<0.5:
        if time%1<0.25:
          if plan_for=='hybrd_ded_simple_heur':
              cc=customers_ic
          else:
              cc=customers_dis
          for cust in cc:
             if cust.tip=='zm':
                if np.max(cust.acc_orders) > 4: cust.dayPassed()
             else:
                if np.max(cust.acc_orders) > 10: cust.dayPassed()


        for i in range(exit_points):
            #Returning the vehicles
            for v in fleets[i].v: #for every vehicle in the fleet of each exit location...
                if not v.returned and (v.return_time < time or plan_for =="spot_only"):#if its returned status is false and return time has already come ...
                    v.stat = "wh"; v.returned = True; fleets[i].v_return(v); v.rem_cap = v.cap #update it as "wh" (in the warehouse) and update fleet with the vehicle (see v_return func. above)
                    if v.tip=="T":
                        trks[i]+=1
                    else:
                        lrs[i]+=1
                    if plan_for != "spot_only":
                        print("#RETURN: ",v.plate, " returned to ", v.base.name) 


            if demand_plan == "frequency_inbnd" :
                for locat in customers_ic:
                    if not locat.index in [7,8,9]:
                        sub_dem = demands_oct.loc[(demands_oct.from_place == customers_ic[wh_ind[i]].name)  &  (demands_oct.to_place == locat.name)]
                        if not sub_dem.empty:
                            if (time-1) % int(30/sub_dem["norm_count"]) == 0 and time != maxdays:
                                if not locat.index in [0,1,2]:
                                    locat.acc_orders[i] += int(sub_dem["norm_periodic"].iloc[0])+1 
                                    locat.passive = False   
                                else: 
                                    customers_ic[locat.index + 7].acc_orders[i] += int(sub_dem["norm_periodic"].iloc[0])+1
                                    customers_ic[locat.index + 7].passive = False
            
            if demand_plan == "frequency_outbnd" :
                for locat in customers_dis:
                      sub_dem = demands_oct.loc[(demands_oct.from_place == customers_dis[wh_ind_outbnd[i]].name)  &  (demands_oct.to_place == locat.name)]
                      if not sub_dem.empty:
                          if (time-1) % int(30/sub_dem["norm_count"]) == 0 and time != maxdays:
                              if not locat.index in [0,1,2]:
                                  locat.acc_orders[i] += int(sub_dem["norm_periodic"].iloc[0])+1 
                                  locat.passive = False   
                                  
            
            fleets[i].avail_check()
            
        if (time+1)%7 != 0 and time%7 !=0 :

            if plan_for == "hybrd_ded_simple_heur":
                possible_depots =[d for d in range(exit_points)]; thresholds = [[105,60,60] if not hyb_inb else [80,45,50, 50,50,50,50]][0]; t=[]
                m=[];
                while len(possible_depots) > 0:
                    [cur_max, chosen_wh_ind, chosen_dest_ind] = load_benefits_start_inbnd(possible_depots,hyb_inb)
                    if cur_max > thresholds[chosen_wh_ind]:###########
                        deal_rts=[[2,7],[0,8],[1,7],[6,8],[8,6],[1,9],[2,8],[0,9]]; 
                        pr_ded_rts=[[0,7],[0,8],[0,9],[1,7],[1,8],[1,9],[2,7],[2,8],[2,9]]
                                                                                                
                        p=1; k=1; w=0; x='None'
                        
                        if (chosen_wh_ind in [3,4,5,6]) and fab_spot:
                            chosen_vehicle = find_vehicle_inbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot=True)
                            p=0; x='Changed'
                        else:
                          while k:
                              for j in pr_ded_rts:
                                  if ([chosen_wh_ind,chosen_dest_ind]==j):
                                      if customers_ic[chosen_dest_ind].acc_orders[chosen_wh_ind]>15:
                                          if ([chosen_wh_ind,chosen_dest_ind]==j) and (trks[chosen_wh_ind]>0):
                                              chosen_vehicle = find_vehicle_inbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot=False)
                                              trks[chosen_wh_ind]-=1
                                              k=0; w=1; x='Changed'
                                              break
                                      else:
                                          if ([chosen_wh_ind,chosen_dest_ind]==j) and (lrs[chosen_wh_ind]>0):
                                              chosen_vehicle = find_vehicle_inbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot=False)
                                              lrs[chosen_wh_ind]-=1
                                              k=0; w=1;  x='Changed'
                                              break
                              break
                          
                          if (1-w):
                            while p:                          
                                for i in deal_rts:
                                    if ([chosen_wh_ind,chosen_dest_ind]==i): #or (chosen_wh_ind in [3,4,5,6]):     
                                        chosen_vehicle = find_vehicle_inbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot=True)
                                        p=0; x='Changed'
                                        break
                                break

                        if (x=='None') or (chosen_vehicle==0):
                            w=0;
                            chosen_vehicle = find_vehicle_inbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot=True)
                        
                        if (w) and (chosen_vehicle != 0): #routes with dedicated trucks
                            strprint = chosen_vehicle.plate +" "+ customers_ic[chosen_wh_ind].name
                            [newroute, distanceOfRoute, strprint, stats] = find_route_inbnd_ded(strprint, chosen_vehicle, chosen_dest_ind, spot=False)
                            route_appender(tot_route,newroute)
                            all_empty_distances[chosen_wh_ind].append(stats.emptyDistRate)
                            totalReturnCost[chosen_wh_ind] += stats.emptyDistRate*distanceOfRoute*chosen_vehicle.cost
                            all_distances[chosen_wh_ind].append(distanceOfRoute)
                            all_mean_fill[chosen_wh_ind].append(stats.mean_fill)
                            all_mean_fill_bin[chosen_wh_ind].append(stats.mean_fill_bin)
                            cost = distanceOfRoute * chosen_vehicle.cost
                            total_cost[chosen_wh_ind] += cost
                            strprint += " -- " + str(distanceOfRoute) + "km" + "(₺" + str(int(cost)) + ")"
                            duration = distanceOfRoute/(chosen_vehicle.speed) + (len(newroute)-2)*1 #LOAD UNLOAD DURATION
                            if duration > 9: duration = 24*int(duration/9) + duration
                            chosen_vehicle.return_time = time + duration/24
                            print(strprint)
                            fleets[chosen_wh_ind].avail_check()
 
                        elif (1-w) and (chosen_vehicle != 0):  #spot routes with deals
                            strprint = chosen_vehicle.plate +" "+ customers_ic[chosen_wh_ind].name
                            [newroute, distanceOfRoute, strprint, stats] = find_route_inbnd_ded(strprint, chosen_vehicle, chosen_dest_ind, spot=True)
                            route_appender(tot_route,newroute)
                            all_empty_distances[chosen_wh_ind].append(stats.emptyDistRate)
                            totalReturnCost[chosen_wh_ind] += stats.emptyDistRate*distanceOfRoute*chosen_vehicle.cost
                            all_distances[chosen_wh_ind].append(distanceOfRoute)
                            all_mean_fill[chosen_wh_ind].append(stats.mean_fill)
                            all_mean_fill_bin[chosen_wh_ind].append(stats.mean_fill_bin)
                            [cost,extraReturnCost] = spot_cost_calculator_hyb_inb(newroute, distanceOfRoute, chosen_vehicle, contracted_routes, contracted_costs)
                            
                            cost = distanceOfRoute * chosen_vehicle.cost
                            total_cost[chosen_wh_ind] += cost
                            totalReturnCost[chosen_wh_ind] += extraReturnCost
                          
                            ########
                            total_cost[chosen_wh_ind] += cost
                            strprint += " -- " + str(distanceOfRoute) + "km" + "(₺" + str(int(cost)) + ")"
                            duration = distanceOfRoute/(chosen_vehicle.speed) + (len(newroute)-2)*1 #LOAD UNLOAD DURATION
                            if duration > 9: duration = 24*int(duration/9) + duration
                            chosen_vehicle.return_time = time + duration/24
                            print(strprint)
                            fleets[chosen_wh_ind].avail_check()
                        else: 
                            if chosen_dest_ind>0:
                              possible_depots.remove(chosen_wh_ind)
                              print("No vehicle found in ", customers_ic[chosen_wh_ind].name)
                              vehicle_avail[chosen_wh_ind].append(fleets[chosen_wh_ind].available_lorry + fleets[chosen_wh_ind].available_truck)
                              m.append(1)
                    else:
                        if chosen_dest_ind>0:
                          possible_depots.remove(chosen_wh_ind)
                          print("Shipments from ", customers_ic[chosen_wh_ind].name, " no longer desirable today.")
                          vehicle_avail[chosen_wh_ind].append(fleets[chosen_wh_ind].available_lorry + fleets[chosen_wh_ind].available_truck)
                        else:
                          break

            elif plan_for == "hybrd_spot_simple_heur":
                possible_depots =[d for d in range(exit_points)]; thresholds = [[105,60,60] if not hyb_inb else [80,45,50, 50,50,50,50]][0];
                locs=[];
                for locat in customers_dis:
                    locat.w_o_truck(woTr)
                    locs.append(locat)
                while len(possible_depots) > 0:
                    [cur_max, chosen_wh_ind, chosen_dest_ind] = load_benefits_start_spot_hyb(possible_depots,hyb_inb)
                    if cur_max > thresholds[chosen_wh_ind]:###########
                        chosen_vehicle = find_vehicle_outbnd(chosen_wh_ind, chosen_dest_ind, lookForSpot=True)
                        if chosen_vehicle != 0:
                            strprint = chosen_vehicle.plate +" "+ customers_dis[wh_ind_outbnd[chosen_wh_ind]].name
                            [newroute, distanceOfRoute, strprint, stats] = find_route_outbnd_spt(strprint, chosen_vehicle, chosen_dest_ind, spot=True)
                            #CONVERT TO SPOT?? CONTINUE WITH ADJUSTED SPOT ROUTE
                            all_empty_distances[chosen_wh_ind].append(stats.emptyDistRate)
                            all_distances[chosen_wh_ind].append(distanceOfRoute)
                            all_mean_fill[chosen_wh_ind].append(stats.mean_fill)
                            all_mean_fill_bin[chosen_wh_ind].append(stats.mean_fill_bin)
                            #distanceOfRoute += data_geb["distance_matrix"][finalpoint][chosen_wh_ind]
                            [cost,extraReturnCost] = spot_cost_calculator_hyb(newroute, distanceOfRoute, chosen_vehicle, contracted_routes, contracted_costs)

                            cost = distanceOfRoute * chosen_vehicle.cost
                            total_cost[chosen_wh_ind] += cost
                            totalReturnCost[chosen_wh_ind] += extraReturnCost

                            total_cost[chosen_wh_ind] += cost
                            strprint += " -- " + str(distanceOfRoute) + "km" + "(₺" + str(int(cost)) + ")"
                            duration = distanceOfRoute/(chosen_vehicle.speed) + (len(newroute)-2)*1 #LOAD UNLOAD DURATION
                            if duration > 9: duration = 24*int(duration/9) + duration%9
                            chosen_vehicle.return_time = time + duration/24
                            print(strprint)
                            fleets[chosen_wh_ind].avail_check()
                        else: 
                            possible_depots.remove(chosen_wh_ind)
                            print("No vehicle found in ", customers_dis[wh_ind_outbnd[chosen_wh_ind]].name)
                            vehicle_avail[chosen_wh_ind].append(fleets[chosen_wh_ind].available_lorry + fleets[chosen_wh_ind].available_truck)
                    else:
                        if chosen_dest_ind>0:
                            possible_depots.remove(chosen_wh_ind)
                            print("Shipments from ", customers_dis[wh_ind[chosen_wh_ind]].name, " no longer desirable today.")
                            vehicle_avail[chosen_wh_ind].append(fleets[chosen_wh_ind].available_lorry + fleets[chosen_wh_ind].available_truck)
                        else: break

    
    else: #weekends
        print("WEEKEND")

    for cust in customers:
        cust.serviceLevel.append(cust.days_since_ship)

weightedFillRates=[]
for d in range(exit_points):  
    weightedFillRates.append(np.inner(all_distances[d],all_mean_fill[d]) / sum(all_distances[d]))

weightedFillRatesBin=[]
for d in range(exit_points):  
    weightedFillRatesBin.append(np.inner(all_distances[d],all_mean_fill_bin[d]) / sum(all_distances[d]))

print(plan_for)
print(total_cost)
print(sum(total_cost))
#print(m)
#left over demand(not served)
leftOver=0
for c in customers:
    leftOver+= sum(c.acc_orders)
print("leftover", leftOver)


### DAY  1 dedike  MORNING ###

### DAY  1 dedike  NOON ###
34 T 01 GebzeDepo -> DogusEsenyurt(given:20) -> Gurpa2(given:13) -- 77.0km(₺216)
34 T 02 GebzeDepo -> PirimIstanbul(given:21) -> Gurpa1(given:12) -- 114.0km(₺320)
34 T 01 AlasehirDepo -> MetroIzmir(given:26) -- 81.0km(₺227)
34 T 03 GebzeDepo -> Gurpa1(given:4) -> Gurpa2(given:4) -- 64.0km(₺179)
Shipments from  AlasehirDepo  no longer desirable today.
WEEKEND
WEEKEND

### DAY  2 dedike  MORNING ###
#RETURN:  34 T 01  returned to  GebzeDepo
#RETURN:  34 T 02  returned to  GebzeDepo
#RETURN:  34 T 03  returned to  GebzeDepo
#RETURN:  34 T 01  returned to  AlasehirDepo
34 T 01 GebzeDepo -> DogusEsenyurt(given:20) -> MahaloGıda(given:7) -- 34.0km(₺95)
34 T 02 GebzeDepo -> GurpaAsya(given:16) -> PirimIstanbul(given:17) -- 35.0km(₺98)
34 T 01 AlasehirDepo -> MetroIzmir(given:26) -> OzdilekEskisehir(given:3) -- 81.0km(₺227)
34 T 03 GebzeDepo -> Gurpa1(given:16) -> DogusCorlu(given:14) -- 43.0km(₺120)
34 T 01 NevsehirDepo -> AltanlarE

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:242: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:246: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
for locats in customers_dis

1

In [ ]:
def route_counter(lst,custs):
  elements=[]; cntr=[];
  for i in lst:
    a=""
    for j in range(len(i)):
      if not j==(len(i)-1):
        a+=str(custs[i[j]].name) + "-" 
      else:
        a+=str(custs[i[j]].name)
    elements.append(a)
  
  for i in range(len(elements)):
    b=""
    p=elements.count(elements[i])
    b+=str(elements[i])+ " " +"->" +"count:"+str(p)
    cntr.append(b)
  return cntr

In [ ]:
route_counter(tot_route,customers_ic)

['AlasehirDepo-GebzeDepo_dem ->count:4',
 'MersinAntrepo-NevsehirDepo_dem ->count:30',
 'BilecikFabrika-GebzeDepo_dem ->count:15',
 'NevsehirDepo-GebzeDepo_dem ->count:9',
 'AlasehirDepo-GebzeDepo_dem-AlasehirDepo ->count:53',
 'BilecikFabrika-AlasehirDepo_dem ->count:30',
 'NevsehirDepo-GebzeDepo_dem ->count:9',
 'AlasehirDepo-GebzeDepo_dem ->count:4',
 'MersinAntrepo-NevsehirDepo_dem ->count:30',
 'NevsehirDepo-GebzeDepo_dem ->count:9',
 'AlasehirDepo-GebzeDepo_dem ->count:4',
 'BilecikFabrika-AlasehirDepo_dem ->count:30',
 'NevsehirDepo-GebzeDepo_dem ->count:9',
 'NevsehirDepo-AlasehirDepo_dem ->count:5',
 'NevsehirDepo-GebzeDepo_dem ->count:9',
 'AlasehirDepo-GebzeDepo_dem-AlasehirDepo ->count:53',
 'MersinAntrepo-NevsehirDepo_dem ->count:30',
 'BilecikFabrika-GebzeDepo_dem ->count:15',
 'BilecikFabrika-AlasehirDepo_dem ->count:30',
 'AlasehirDepo-GebzeDepo_dem-AlasehirDepo ->count:53',
 'NevsehirDepo-GebzeDepo_dem-NevsehirDepo ->count:58',
 'NevsehirDepo-AlasehirDepo_dem-NevsehirD

In [ ]:
customers_ic[5].name

'MersinAntrepo'

In [ ]:
tot_route

[[2, 7],
 [5, 8],
 [4, 7],
 [1, 7],
 [2, 7, 2],
 [4, 9],
 [1, 7],
 [2, 7],
 [5, 8],
 [1, 7],
 [2, 7],
 [4, 9],
 [1, 7],
 [1, 9],
 [1, 7],
 [2, 7, 2],
 [5, 8],
 [4, 7],
 [4, 9],
 [2, 7, 2],
 [1, 7, 1],
 [1, 9, 1],
 [1, 7],
 [2, 7, 2],
 [5, 8],
 [3, 7],
 [1, 7, 1],
 [4, 9],
 [2, 7],
 [1, 7, 1],
 [1, 9, 1],
 [2, 8],
 [2, 7, 2],
 [5, 8],
 [0, 8],
 [4, 7],
 [1, 7, 1],
 [4, 9],
 [2, 7, 2],
 [1, 7, 1],
 [4, 8],
 [1, 9],
 [5, 8],
 [1, 7],
 [2, 7, 2],
 [0, 8],
 [4, 9],
 [1, 7, 1],
 [2, 7, 2],
 [1, 9, 1],
 [2, 8, 2],
 [5, 8],
 [1, 7],
 [0, 9, 0],
 [2, 7, 2],
 [4, 7],
 [0, 8],
 [3, 7],
 [4, 9],
 [1, 7, 1],
 [2, 7, 2],
 [1, 9, 1],
 [4, 8],
 [1, 7, 1],
 [5, 8],
 [2, 7, 2],
 [0, 8, 0],
 [1, 7, 1],
 [4, 9],
 [2, 7, 2],
 [1, 9, 1],
 [1, 7, 1],
 [2, 8],
 [5, 8],
 [1, 7],
 [1, 9],
 [4, 7],
 [0, 9, 0],
 [2, 7, 2],
 [0, 8, 0],
 [4, 9],
 [1, 7, 1],
 [2, 7, 2],
 [4, 8],
 [5, 8],
 [1, 7, 1],
 [1, 9],
 [2, 7, 2],
 [3, 7],
 [0, 8],
 [4, 9],
 [1, 7, 1],
 [2, 7, 2],
 [2, 8, 2],
 [5, 8],
 [1, 7, 1],
 [1, 9, 1],
 